#Environment setup (ADK, key, no Vertex)

In [88]:
# A3-1: Environment prep (ADK + key + no Vertex)

import os, sys, subprocess

print("Python:", sys.version)

# 1) Install deps (idempotent)
!pip -q install --upgrade google-adk requests pandas

# 2) Set your AI Studio key (same pattern as A1/A2). If your env already has it, we keep it.
os.environ["GOOGLE_API_KEY"] = os.environ.get("GOOGLE_API_KEY", "")
print("GOOGLE_API_KEY (masked):", (os.environ["GOOGLE_API_KEY"][:6] + "****") if os.environ["GOOGLE_API_KEY"] else "<missing>")

# 3) Make sure we don't route to Vertex
for var in ["GOOGLE_GENAI_USE_VERTEXAI", "GOOGLE_VERTEX_PROJECT", "GOOGLE_VERTEX_LOCATION", "GOOGLE_CLOUD_PROJECT"]:
    os.environ.pop(var, None)
print("Vertex-related env disabled.")

# 4) Quick CLI sanity
try:
    out = subprocess.run(["adk", "--version"], check=False, capture_output=True, text=True)
    if out.returncode == 0 and out.stdout.strip():
        print("adk --version:", out.stdout.strip())
    else:
        help_out = subprocess.run(["adk", "--help"], check=False, capture_output=True, text=True)
        print("adk --help (first lines):\n", "\n".join(help_out.stdout.splitlines()[:5]))
except FileNotFoundError:
    print("ERROR: `adk` CLI not found on PATH")


Python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
GOOGLE_API_KEY (masked): AIzaSy****
Vertex-related env disabled.
adk --version: adk, version 1.16.0


#Create and seed the travel SQLite database

In [89]:
# A3-2: Create a clean SQLite DB with consistent column names

import sqlite3, os

db_path = "travel.db"
if os.path.exists(db_path):
    os.remove(db_path)

conn = sqlite3.connect(db_path)
cur = conn.cursor()

cur.executescript("""
DROP TABLE IF EXISTS flights;
DROP TABLE IF EXISTS hotels;
DROP TABLE IF EXISTS attractions;

CREATE TABLE flights (
  flight_no   TEXT PRIMARY KEY,
  carrier     TEXT,
  origin      TEXT,
  dest        TEXT,
  depart      TEXT,
  arrive      TEXT,
  price       REAL
);

CREATE TABLE hotels (
  id INTEGER PRIMARY KEY,
  name TEXT,
  city TEXT,
  neighborhood TEXT,
  price REAL,
  rating REAL
);

CREATE TABLE attractions (
  id INTEGER PRIMARY KEY,
  name TEXT,
  city TEXT,
  category TEXT,
  hours TEXT,
  popularity INTEGER
);
""")

flights = [
  ("JL001","JAL","SFO","NRT","08:30","12:20", 780.0),
  ("NH007","ANA","SFO","HND","10:00","14:15", 820.0),
  ("UA837","United","SFO","NRT","12:00","15:45", 695.0),
  ("SQ012","Singapore","LAX","NRT","09:15","13:10", 720.0),
]

hotels = [
  (1,"Park Hotel Tokyo","Tokyo","Shiodome",180,4.3),
  (2,"Hotel Gracery Shinjuku","Tokyo","Shinjuku",150,4.2),
  (3,"The Gate Hotel Asakusa","Tokyo","Asakusa",135,4.1),
  (4,"Kyoto Granbell Hotel","Kyoto","Gion",160,4.4),
  (5,"Hotel Mystays Kyoto","Kyoto","Central",120,4.0),
]

attractions = [
  (1,"Senso-ji Temple","Tokyo","temple","06:00-17:00",95),
  (2,"Ueno Park","Tokyo","park","All day",88),
  (3,"Meiji Jingu","Tokyo","shrine","Sunrise-Sunset",92),
  (4,"Kiyomizu-dera","Kyoto","temple","06:00-18:00",97),
  (5,"Fushimi Inari Taisha","Kyoto","shrine","All day",99),
]

cur.executemany("INSERT INTO flights VALUES (?,?,?,?,?,?,?)", flights)
cur.executemany("INSERT INTO hotels  VALUES (?,?,?,?,?,?)", hotels)
cur.executemany("INSERT INTO attractions VALUES (?,?,?,?,?,?)", attractions)
conn.commit()

print("Rows — flights, hotels, attractions:",
      cur.execute("SELECT COUNT(*) FROM flights").fetchone()[0],
      cur.execute("SELECT COUNT(*) FROM hotels").fetchone()[0],
      cur.execute("SELECT COUNT(*) FROM attractions").fetchone()[0])

# Quick peek
print(cur.execute("SELECT flight_no, carrier, origin, dest, price FROM flights ORDER BY price ASC").fetchall()[:3])

conn.close()


Rows — flights, hotels, attractions: 4 5 5
[('UA837', 'United', 'SFO', 'NRT', 695.0), ('SQ012', 'Singapore', 'LAX', 'NRT', 720.0), ('JL001', 'JAL', 'SFO', 'NRT', 780.0)]


#Build the travel agent package (FunctionTool + SQLite)

In [90]:
# A3-3: Build a3_travel_agent with 3 FunctionTool-backed SQL helpers

import os, pathlib, sqlite3, json, re, subprocess, textwrap

# Make sure the subprocess (adk run) can find both the key and DB
os.environ["TRAVEL_DB_PATH"] = os.path.abspath("travel.db")
print("TRAVEL_DB_PATH:", os.environ["TRAVEL_DB_PATH"])

# Fresh package
!rm -rf a3_travel_agent
!mkdir -p a3_travel_agent

# __init__.py
open("a3_travel_agent/__init__.py","w").write("from .agent import root_agent\n")

# db_tools.py — plain Python functions that query SQLite
open("a3_travel_agent/db_tools.py","w").write(textwrap.dedent(f"""
import os, sqlite3
from typing import List, Dict, Optional

DB_PATH = os.getenv("TRAVEL_DB_PATH", "{os.path.abspath('travel.db')}")

def _rows(sql: str, params: tuple = ()) -> List[Dict]:
    conn = sqlite3.connect(DB_PATH)
    conn.row_factory = sqlite3.Row
    cur = conn.cursor()
    cur.execute(sql, params)
    out = [dict(r) for r in cur.fetchall()]
    conn.close()
    return out

def hotels_by_city(city: str, max_price: Optional[float] = None, limit: int = 5) -> List[Dict]:
    \"\"\"Return hotels in a city, optional max_price filter.
    Args:
      city: e.g., "Tokyo"
      max_price: USD max nightly price (optional)
      limit: number of rows to return (default 5)
    \"\"\"
    sql = "SELECT name, neighborhood, price FROM hotels WHERE city = ?"
    params = [city]
    if max_price is not None:
        sql += " AND price <= ?"
        params.append(float(max_price))
    sql += " ORDER BY price ASC LIMIT ?"
    params.append(int(limit))
    return _rows(sql, tuple(params))

def attractions_by_city(city: str, category: Optional[str] = None, limit: int = 5) -> List[Dict]:
    \"\"\"Return top attractions in a city (by popularity), optional category filter.
    Args:
      city: e.g., "Kyoto"
      category: e.g., "temple" (optional)
      limit: number of rows to return (default 5)
    \"\"\"
    sql = "SELECT name, category, hours FROM attractions WHERE city = ?"
    params = [city]
    if category:
        sql += " AND category = ?"
        params.append(category)
    sql += " ORDER BY popularity DESC LIMIT ?"
    params.append(int(limit))
    return _rows(sql, tuple(params))

def flights_search(origin: str, dest: str, max_price: Optional[float] = None, limit: int = 5) -> List[Dict]:
    \"\"\"Return best-priced flights for an origin→dest (IATA codes), optional max_price.
    Args:
      origin: e.g., "SFO"
      dest:   e.g., "NRT"
      max_price: USD cap (optional)
      limit: max rows (default 5)
    \"\"\"
    sql = "SELECT carrier, flight_no, depart, arrive, price FROM flights WHERE origin = ? AND dest = ?"
    params = [origin, dest]
    if max_price is not None:
        sql += " AND price <= ?"
        params.append(float(max_price))
    sql += " ORDER BY price ASC LIMIT ?"
    params.append(int(limit))
    return _rows(sql, tuple(params))
"""))

# agent.py — expose the functions as ADK tools
open("a3_travel_agent/agent.py","w").write(textwrap.dedent("""
from google.adk.agents import Agent
from google.adk.tools import FunctionTool

from .db_tools import hotels_by_city, attractions_by_city, flights_search

root_agent = Agent(
    model="gemini-2.0-flash",
    name="a3_travel_agent",
    description="Travel agent that queries local SQLite for hotels/attractions/flights.",
    instruction=(
        "You can call tools to query hotels, attractions, and flights. "
        "Prefer tools for factual lookups; answer concisely. "
        "If a user asks for hotels, call hotels_by_city; for sights, attractions_by_city; "
        "for flights, flights_search."
    ),
    tools=[
        FunctionTool(hotels_by_city),
        FunctionTool(attractions_by_city),
        FunctionTool(flights_search),
    ],
)
"""))

# .env for the package so `adk run a3_travel_agent` inherits config
open("a3_travel_agent/.env","w").write(
    f"GOOGLE_API_KEY={os.environ['GOOGLE_API_KEY']}\n"
    f"TRAVEL_DB_PATH={os.environ['TRAVEL_DB_PATH']}\n"
)

print("[OK] a3_travel_agent built with FunctionTool-backed SQLite queries")
!ls -la a3_travel_agent


TRAVEL_DB_PATH: /content/travel.db
[OK] a3_travel_agent built with FunctionTool-backed SQLite queries
total 24
drwxr-xr-x 2 root root 4096 Oct 20 05:15 .
drwxr-xr-x 1 root root 4096 Oct 20 05:15 ..
-rw-r--r-- 1 root root  755 Oct 20 05:15 agent.py
-rw-r--r-- 1 root root 2271 Oct 20 05:15 db_tools.py
-rw-r--r-- 1 root root   89 Oct 20 05:15 .env
-rw-r--r-- 1 root root   30 Oct 20 05:15 __init__.py


#Quick sanity tests for hotels, attractions, and flights tools

In [91]:
import os, re, subprocess, textwrap, json

def run_agent(agent_pkg: str, prompt: str):
    p = subprocess.run(
        ["adk","run",agent_pkg],
        input=(prompt+"\n").encode(),
        stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
        check=False, env=dict(os.environ),
    )
    out = p.stdout.decode()
    m = re.search(rf"\[{re.escape(agent_pkg)}\]:\s*(.*)", out, flags=re.S)
    return (m.group(1).strip() if m else out), out

print(">>> Hotels")
hotels_ans, hotels_raw = run_agent(
    "a3_travel_agent",
    "Use the hotels_by_city tool with city='Tokyo', max_price=170, limit=3. "
    "Return 'name — neighborhood — $price' bullets."
)
print(hotels_ans, "\n")

print(">>> Attractions")
attr_ans, attr_raw = run_agent(
    "a3_travel_agent",
    "Use attractions_by_city with city='Kyoto', category='temple', limit=3. "
    "Return 'name — hours' bullets."
)
print(attr_ans, "\n")

print(">>> Flights")
flights_ans, flights_raw = run_agent(
    "a3_travel_agent",
    "Use flights_search with origin='SFO', dest='NRT', max_price=800, limit=3. "
    "Return 'carrier flight_no — depart→arrive — $price' bullets."
)
print(flights_ans, "\n")

# Optional: peek at tool calls (if the log captured them)
print(">>> Recent tool calls (if logged)")
try:
    log_tail = subprocess.run(
        ["bash","-lc","tail -n 200 /tmp/agents_log/agent.latest.log | sed -n '/functionCall/,$p'"],
        stdout=subprocess.PIPE, stderr=subprocess.STDOUT, check=False
    ).stdout.decode()
    print(log_tail or "(no functionCall records in tail)")
except Exception as e:
    print("(could not read log tail)", e)


>>> Hotels
* The Gate Hotel Asakusa — Asakusa — $135
* Hotel Gracery Shinjuku — Shinjuku — $150
[user]: 
Aborted! 

>>> Attractions
* Kiyomizu-dera — 06:00-18:00

[user]: 
Aborted! 

>>> Flights
* United UA837 — 12:00→15:45 — $695
* JAL JL001 — 08:30→12:20 — $780
[user]: 
Aborted! 

>>> Recent tool calls (if logged)
(no functionCall records in tail)


#Install trip-planner policy (overwrite agent instruction)

In [92]:
%%bash
set -e
cat > a3_travel_agent/agent.py << 'PY'
from google.adk.agents import Agent
from google.adk.tools import FunctionTool
from .db_tools import hotels_by_city, attractions_by_city, flights_search

root_agent = Agent(
    model="gemini-2.0-flash",
    name="a3_travel_agent",
    description="Trip planner that queries a local SQLite travel DB via tools.",
    instruction=(
        "You are a travel planner. IMPORTANT:\n"
        "• For any request about hotels, flights, or attractions, you MUST call the corresponding tool(s):\n"
        "  - hotels_by_city(city, max_price?, limit?)\n"
        "  - flights_search(origin, dest, max_price?, limit?)\n"
        "  - attractions_by_city(city, category?, limit?)\n"
        "• Never invent data. Only cite rows returned by the tools. If a tool returns no rows, say so.\n"
        "• When asked to 'plan' a trip, do this sequence:\n"
        "  1) Find flights (cheapest first) with flights_search.\n"
        "  2) Pick 1–2 hotels via hotels_by_city under the given budget (if provided).\n"
        "  3) List 3–5 attractions via attractions_by_city.\n"
        "  4) Produce a concise 1–2 day itinerary: bullets per day.\n"
        "• Output format for planning:\n"
        "  - 'Flights:' one line per option as 'Carrier Flight — depart→arrive — $price' (max 3 lines).\n"
        "  - 'Hotels:' 'Name — neighborhood — $price' (max 3 lines).\n"
        "  - 'Attractions:' 3–5 bullets 'Name — hours'.\n"
        "  - 'Itinerary:' Day 1 / Day 2 bullets using the above.\n"
        "• End with: 'Source: travel.db'."
    ),
    tools=[
        FunctionTool(hotels_by_city),
        FunctionTool(attractions_by_city),
        FunctionTool(flights_search),
    ],
)
PY
echo "[OK] Planner policy installed"


[OK] Planner policy installed


#Generate a 2-day Tokyo plan via tools

In [93]:
import os, re, subprocess

def run_agent(agent_pkg: str, prompt: str):
    p = subprocess.run(
        ["adk","run",agent_pkg],
        input=(prompt+"\n").encode(),
        stdout=subprocess.PIPE, stderr=subprocess.STDOUT, check=False, env=dict(os.environ)
    )
    out = p.stdout.decode()
    m = re.search(rf"\[{re.escape(agent_pkg)}\]:\s*(.*)", out, flags=re.S)
    return (m.group(1).strip() if m else out), out

plan_prompt = (
    "Plan a 2-day weekend in Tokyo for someone flying from SFO to NRT under $800 "
    "and staying under $170/night. Use the tools, then build the plan."
)
ans, raw = run_agent("a3_travel_agent", plan_prompt)
print(ans)


OK. I have planned a trip to Tokyo for you.

Flights:
*   United UA837 — 12:00→15:45 — $695
*   JAL JL001 — 08:30→12:20 — $780

Hotels:
*   The Gate Hotel Asakusa — Asakusa — $135
*   Hotel Gracery Shinjuku — Shinjuku — $150

Attractions:
*   Senso-ji Temple — 06:00-17:00
*   Meiji Jingu — Sunrise-Sunset
*   Ueno Park — All day

Itinerary:
*   Day 1:
    *   Arrive in Tokyo and check into The Gate Hotel Asakusa.
    *   Visit Senso-ji Temple in the afternoon.
    *   Explore Ueno Park in the evening.
*   Day 2:
    *   Visit Meiji Jingu in the morning.
    *   Explore the Shinjuku area, where the Hotel Gracery Shinjuku is located.

Source: travel.db

[user]: 
Aborted!


#Inspect database schema (PRAGMA table_info)

In [94]:
import sqlite3, os
db = os.path.abspath("travel.db")
con = sqlite3.connect(db)
cur = con.cursor()
for t in ("hotels","attractions","flights"):
    cols = [c[1] for c in cur.execute(f"PRAGMA table_info({t})").fetchall()]
    print(f"{t}: {cols}")
con.close()


hotels: ['id', 'name', 'city', 'neighborhood', 'price', 'rating']
attractions: ['id', 'name', 'city', 'category', 'hours', 'popularity']
flights: ['flight_no', 'carrier', 'origin', 'dest', 'depart', 'arrive', 'price']


#Update db_tools.py to return formatted strings

In [95]:
# Overwrite a3_travel_agent/db_tools.py with formatted-string tools (match CURRENT schema)
import os, textwrap, pathlib

pkg = pathlib.Path("a3_travel_agent")
assert (pkg / "__init__.py").exists(), "a3_travel_agent package not found — re-run your A3-3 build cell first."

code = """
# A3: fix tools to match CURRENT DB schema
import os, sqlite3
from typing import List, Dict, Optional

DB_PATH = os.getenv("TRAVEL_DB_PATH", os.path.abspath("travel.db"))

def _rows(sql: str, params: tuple = ()) -> List[Dict]:
    con = sqlite3.connect(DB_PATH)
    con.row_factory = sqlite3.Row
    cur = con.cursor()
    cur.execute(sql, params)
    out = [dict(r) for r in cur.fetchall()]
    con.close()
    return out

def hotels_by_city(city: str, max_price: Optional[float] = None, limit: int = 5) -> str:
    \"\"\"
    Uses hotels.price (CURRENT schema). Returns a formatted string with header + Source.
    \"\"\"
    sql = \"""
    SELECT name, neighborhood, price
    FROM hotels
    WHERE city = ?
      AND ( ? IS NULL OR price <= ? )
    ORDER BY price ASC
    LIMIT ?
    \"""
    rows = _rows(sql, (city, max_price, max_price, int(limit)))
    lines = [f"*   {r['name']} — {r['neighborhood']} — ${int(r['price'])}" for r in rows]
    header = "Hotels:" if rows else "Hotels: (none found)"
    return "\\n".join([header, *lines, "Source: travel.db"])

def attractions_by_city(city: str, category: Optional[str] = None, limit: int = 5) -> str:
    \"\"\"
    Uses attractions.hours (CURRENT schema). Returns a formatted string with header + Source.
    \"\"\"
    sql = \"""
    SELECT name, hours
    FROM attractions
    WHERE city = ?
      AND ( ? IS NULL OR category = ? )
    ORDER BY popularity DESC
    LIMIT ?
    \"""
    rows = _rows(sql, (city, category, category, int(limit)))
    lines = [f"*   {r['name']} — {r['hours']}" for r in rows]
    header = "Attractions:" if rows else "Attractions: (none found)"
    return "\\n".join([header, *lines, "Source: travel.db"])

def flights_search(origin: str, dest: str, max_price: Optional[float] = None, limit: int = 5) -> str:
    \"\"\"
    Uses flights.depart / flights.arrive (CURRENT schema). Returns a formatted string with header + Source.
    \"\"\"
    sql = \"""
    SELECT carrier, flight_no, depart, arrive, price
    FROM flights
    WHERE origin = ?
      AND dest = ?
      AND ( ? IS NULL OR price <= ? )
    ORDER BY price ASC
    LIMIT ?
    \"""
    rows = _rows(sql, (origin, dest, max_price, max_price, int(limit)))
    lines = [f"*   {r['carrier']} {r['flight_no']} — {r['depart']}→{r['arrive']} — ${int(r['price'])}" for r in rows]
    header = "Flights:" if rows else "Flights: (none found)"
    return "\\n".join([header, *lines, "Source: travel.db"])
"""

(dbtools := (pkg / "db_tools.py")).write_text(textwrap.dedent(code))
print("[OK] Updated", dbtools)


[OK] Updated a3_travel_agent/db_tools.py


#Enforce strict verbatim output from single-tool queries

In [96]:
# A3: strict output policy — return tool string verbatim for single-tool queries
from pathlib import Path
agent_py = Path("a3_travel_agent/agent.py")

agent_py.write_text("""from google.adk.agents import Agent
from google.adk.tools import FunctionTool
from .db_tools import hotels_by_city, attractions_by_city, flights_search

STRICT_POLICY = (
    "STRICT OUTPUT POLICY:\\n"
    "• If the user asks for hotels, attractions, or flights (a single lookup), you MUST call the matching tool\\n"
    "  and reply with the tool's returned string **exactly** — no extra words, no preface, no markdown fences.\\n"
    "  Do NOT add 'OK', 'Here are', or any commentary.\\n"
    "• The tool strings already include the header and 'Source: travel.db'. Return them verbatim.\\n"
    "• If the user asks to 'plan' a trip (needing multiple lookups), you may call multiple tools and then format\\n"
    "  the result using the planning template, ending with 'Source: travel.db'.\\n"
    "• Never invent data; only cite rows returned by tools. If a tool returns no rows, return its '(none found)' header.\\n"
)

root_agent = Agent(
    model="gemini-2.0-flash",
    name="a3_travel_agent",
    description="Travel agent using SQLite tools.",
    instruction=STRICT_POLICY,
    tools=[
        FunctionTool(hotels_by_city),
        FunctionTool(attractions_by_city),
        FunctionTool(flights_search),
    ],
)
""")
print("[OK] Rewrote a3_travel_agent/agent.py with strict output policy")


[OK] Rewrote a3_travel_agent/agent.py with strict output policy


#Regression tests for travel agent

In [97]:
# A3-6: Deterministic regression tests for a3_travel_agent

import os, re, subprocess, textwrap

def run_agent(agent_pkg: str, prompt: str):
    p = subprocess.run(
        ["adk","run",agent_pkg],
        input=(prompt+"\n").encode(),
        stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
        check=False, env=dict(os.environ),
    )
    out = p.stdout.decode()
    m = re.search(rf"\[{re.escape(agent_pkg)}\]:\s*(.*)", out, flags=re.S)
    return (m.group(1).strip() if m else out), out

tests = [
    (
        "Hotels under $170 in Tokyo (max 3)",
        "Use the hotels_by_city tool with city='Tokyo', max_price=170, limit=3. "
        "Return the tool result verbatim.",
        [r"\bHotels:", r"Source:\s*travel\.db"]
    ),
    (
        "Kyoto temples (max 3)",
        "Use attractions_by_city with city='Kyoto', category='temple', limit=3. "
        "Return the tool result verbatim.",
        [r"\bAttractions:", r"Source:\s*travel\.db"]
    ),
    (
        "Flights SFO→NRT under $800 (max 3)",
        "Use flights_search with origin='SFO', dest='NRT', max_price=800, limit=3. "
        "Return the tool result verbatim.",
        [r"\bFlights:", r"Source:\s*travel\.db"]
    ),
]

for i, (title, prompt, expects) in enumerate(tests, 1):
    ans, raw = run_agent("a3_travel_agent", prompt)
    print(f"\n--- Test {i}: {title}\n{ans[:600]}{'...' if len(ans)>600 else ''}\n")
    missing = [pat for pat in expects if not re.search(pat, ans, re.I | re.S)]
    assert not missing, f"Failed expectations: {missing}"

print("\nAll A3-6 tests passed ✅")

# Optional: peek at recent tool calls, if available in the ADK log
try:
    log_tail = subprocess.run(
        ["bash","-lc","tail -n 200 /tmp/agents_log/agent.latest.log | sed -n '/functionCall/,$p'"],
        stdout=subprocess.PIPE, stderr=subprocess.STDOUT, check=False
    ).stdout.decode()
    if log_tail.strip():
        print("\n--- Recent tool calls (from ADK log) ---\n", log_tail)
except Exception as e:
    pass



--- Test 1: Hotels under $170 in Tokyo (max 3)
Hotels:
*   The Gate Hotel Asakusa — Asakusa — $135
*   Hotel Gracery Shinjuku — Shinjuku — $150
Source: travel.db

[user]: 
Aborted!


--- Test 2: Kyoto temples (max 3)
Attractions:
* Kiyomizu-dera — 06:00-18:00
Source: travel.db

[user]: 
Aborted!


--- Test 3: Flights SFO→NRT under $800 (max 3)
Flights:
*   United UA837 — 12:00→15:45 — $695
*   JAL JL001 — 08:30→12:20 — $780
Source: travel.db

[user]: 
Aborted!


All A3-6 tests passed ✅
